In [3]:
!pip install ortools

     |████████████████████████████████| 14.4MB 249kB/s 
     |████████████████████████████████| 1.0MB 41.3MB/s 
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4


tutaj przypadkowe dystanse, pickup/delivery points i demands

In [71]:
import numpy as np
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

np.random.seed(0)
n=50 #num of clients


N=[i for i in range(1,n+1)] #clients
V=[0]+N #verticles with central point
x=np.random.rand(len(V))*100
y=np.random.rand(len(V))*100

p=[(i,j) for i in V for j in V ] #all posible routes  
d={(i,j): int(np.hypot([x[i]-x[j]],[y[i]-y[j]])[0]) for (i,j) in p}

distmat=list()

for i in range(n+1):
  distances=list()
  for j in range(n+1):
    distances.append(d[(i,j)])
  distmat.append(distances)

print(distmat)

indexes=np.arange(1,n+1)
np.random.shuffle(indexes) #randomize indexes
pickups=indexes[0:int((n+1)/2)] #cut indexes in half to create random pickup and delivery points
deliveries=indexes[int((n+1)/2):(n+1)]

pnd=list()
for i in range(len(pickups)):
  pnd.append([pickups[i],deliveries[i]])

print(pnd)

demands=np.zeros(len(pickups)*2+1)

for i in range(len(pickups)):
  demands[pickups[i]]+=np.random.randint(1,4)
  demands[deliveries[i]]+=(demands[pickups[i]]*(-1))
print(demands)
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = distmat
    data['pickups_deliveries'] = pnd
    data['demands'] = demands
    data['vehicle_capacities'] = [3,3,3,3,3,3,3,3]
    data['num_vehicles'] = 8
    data['depot'] = 0
    return data

[[0, 57, 34, 22, 30, 23, 21, 34, 45, 32, 40, 21, 30, 44, 48, 59, 62, 48, 41, 62, 43, 59, 18, 37, 58, 18, 51, 42, 32, 18, 28, 43, 27, 12, 55, 10, 35, 15, 62, 17, 29, 32, 31, 51, 28, 19, 53, 57, 49, 30, 16], [57, 0, 89, 79, 87, 34, 78, 55, 78, 89, 88, 38, 86, 81, 89, 64, 113, 19, 89, 15, 58, 8, 45, 25, 112, 70, 103, 72, 89, 73, 72, 92, 39, 44, 100, 47, 90, 42, 23, 67, 47, 90, 27, 95, 80, 40, 109, 60, 106, 46, 73], [34, 89, 0, 12, 18, 55, 22, 46, 38, 22, 18, 55, 5, 33, 59, 88, 58, 77, 17, 91, 52, 89, 52, 66, 48, 18, 45, 39, 8, 28, 46, 17, 60, 46, 60, 42, 1, 47, 89, 23, 61, 16, 62, 57, 10, 48, 40, 86, 30, 62, 17], [22, 79, 12, 0, 13, 45, 11, 43, 42, 16, 26, 44, 7, 38, 50, 76, 53, 69, 25, 83, 50, 80, 40, 58, 45, 12, 41, 40, 9, 16, 34, 27, 49, 35, 52, 32, 13, 37, 82, 17, 49, 13, 52, 49, 12, 39, 38, 74, 30, 50, 6], [30, 87, 18, 13, 0, 53, 9, 55, 54, 4, 37, 50, 14, 50, 40, 74, 40, 79, 36, 92, 63, 89, 44, 67, 32, 24, 28, 53, 10, 15, 29, 36, 53, 43, 41, 41, 20, 45, 92, 30, 51, 3, 61, 38, 24, 49,

tutaj zostawiam odgórny model dla porównania 

In [11]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [
        [
            0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354,
            468, 776, 662
        ],
        [
            548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674,
            1016, 868, 1210
        ],
        [
            776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164,
            1130, 788, 1552, 754
        ],
        [
            696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822,
            1164, 560, 1358
        ],
        [
            582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708,
            1050, 674, 1244
        ],
        [
            274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628,
            514, 1050, 708
        ],
        [
            502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856,
            514, 1278, 480
        ],
        [
            194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320,
            662, 742, 856
        ],
        [
            308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662,
            320, 1084, 514
        ],
        [
            194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388,
            274, 810, 468
        ],
        [
            536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764,
            730, 388, 1152, 354
        ],
        [
            502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114,
            308, 650, 274, 844
        ],
        [
            388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194,
            536, 388, 730
        ],
        [
            354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0,
            342, 422, 536
        ],
        [
            468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536,
            342, 0, 764, 194
        ],
        [
            776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274,
            388, 422, 764, 0, 798
        ],
        [
            662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730,
            536, 194, 798, 0
        ],
    ]
    data['pickups_deliveries'] = [
        [1, 6],
        [2, 10],
        [4, 3],
        [5, 9],
        [7, 8],
        [15, 11],
        [13, 12],
        [16, 14],
    ]
    data['demands'] = [0,1, 2, -1, 1, 1, -1, 1, -1, -1, -2, -2, -1, 1, -1, 2, 1]
    data['vehicle_capacities'] = [3, 4, 3, 4]
    data['num_vehicles'] = 4
    data['depot'] = 0
    return data
                                    

In [72]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))

main pickup n delivery z capacity constraints (chwilowo wyłączyłam uciekanie z lokalnych minimów, jest zakomentowane pod search parameters)

In [73]:
def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Define cost of each arc.
    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Define Transportation Requests.
    for request in data['pickups_deliveries']:
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        routing.solver().Add(
            routing.VehicleVar(pickup_index) == routing.VehicleVar(
                delivery_index))
        routing.solver().Add(
            distance_dimension.CumulVar(pickup_index) <=
            distance_dimension.CumulVar(delivery_index))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

    #search_parameters.local_search_metaheuristic = (
        #routing_enums_pb2.LocalSearchMetaheuristic.SIMULATED_ANNEALING)
    #search_parameters.time_limit.FromSeconds(300)
    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == '__main__':
    main()

Objective: 30665
Route for vehicle 0:
 0 Load(0.0) ->  34 Load(2.0) ->  14 Load(0.0) ->  30 Load(2.0) ->  3 Load(0.0) ->  2 Load(3.0) ->  20 Load(0.0) ->  13 Load(2.0) ->  18 Load(3.0) ->  50 Load(2.0) ->  29 Load(0.0) ->  0 Load(0.0)
Distance of the route: 288m
Load of the route: 0.0

Route for vehicle 1:
 0 Load(0.0) ->  11 Load(1.0) ->  49 Load(3.0) ->  41 Load(1.0) ->  48 Load(0.0) ->  24 Load(2.0) ->  43 Load(3.0) ->  32 Load(2.0) ->  22 Load(0.0) ->  0 Load(0.0)
Distance of the route: 236m
Load of the route: 0.0

Route for vehicle 2:
 0 Load(0.0) ->  45 Load(3.0) ->  9 Load(0.0) ->  28 Load(1.0) ->  36 Load(3.0) ->  44 Load(2.0) ->  8 Load(3.0) ->  17 Load(1.0) ->  40 Load(0.0) ->  0 Load(0.0)
Distance of the route: 272m
Load of the route: 0.0

Route for vehicle 3:
 0 Load(0.0) ->  10 Load(3.0) ->  15 Load(0.0) ->  47 Load(1.0) ->  38 Load(0.0) ->  0 Load(0.0)
Distance of the route: 288m
Load of the route: 0.0

Route for vehicle 4:
 0 Load(0.0) ->  7 Load(3.0) ->  33 Load(0.0) ->